# Preparation

In [22]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

In [23]:
pd.__version__

'0.20.3'

In [24]:
np.__version__

'1.13.1'

### common variables

In [25]:
checkpointsStem = "tutorial1.Checkpoint";
# used in Static data/Google form correct answers.ipynb'
language = 'English'
#language = 'French'

### columns

In [26]:
# In RedMetrics data, 'playerId' is actually a session ID.
# Permanent player IDs are stored as 'localplayerguid' in 'customdata' attached to 'start' events.
minimalInitializationColumns = ['customData.localplayerguid']
initializationColumns = ['customData.localplayerguid', 'playerId', 'type', 'serverTime', 'customData.platform']
relevantColumns = ['sessionId', 'serverTime', 'userId', 'customData.platform']
playerFilteringColumns = ['sessionId', 'userId', 'customData.platform', 'serverTime']
checkpointsRelevantColumns = ['sessionId', 'userId', 'type', 'section']

### dataframes for the different game versions

In [27]:
def getNormalizedRedMetricsCSV( df ):
    newColumns = np.unique(np.concatenate((minimalInitializationColumns, df.columns.values)))
    return safeGetNormalizedRedMetricsCSV(df.loc[:,newColumns])

def safeGetNormalizedRedMetricsCSV( df ):
    return df.rename(index=str, columns={'customData.localplayerguid' : 'userId', 'playerId': 'sessionId'})

In [28]:
rdf100 = pd.read_csv("../../data/1.0.csv")
rdf131 = pd.read_csv("../../data/1.31.csv")
rdf132 = pd.read_csv("../../data/1.32.csv")
rdf133 = pd.read_csv("../../data/1.33.csv")
rdf140 = pd.read_csv("../../data/1.40.csv")
rdf150 = pd.read_csv("../../data/1.50.csv")
rdf151 = pd.read_csv("../../data/1.51.csv")
rdf152 = pd.read_csv("../../data/2017-09-05.1.52.csv")

relevantColumns = ['customData.localplayerguid', 'playerId', 'type']
renamedRelevantColumns = ['userId', 'sessionId', 'type']

part100 = rdf100.loc[:,relevantColumns]
part131 = rdf131.loc[:,relevantColumns]
part132 = rdf132.loc[:,relevantColumns]
part133 = rdf133.loc[:,relevantColumns]
part140 = rdf140.loc[:,relevantColumns]
part150 = rdf150.loc[:,relevantColumns]
part151 = rdf151.loc[:,relevantColumns]
part152 = rdf152.loc[:,relevantColumns]

dfconcat = pd.concat([part100, 
                      part131, part132, part133, 
                      part140, 
                      part150, part151, part152])

df = getNormalizedRedMetricsCSV(dfconcat)
df152 = getNormalizedRedMetricsCSV(rdf152)

rdf = getNormalizedRedMetricsCSV(pd.concat([rdf100,
                  rdf131, rdf132, rdf133,
                  rdf140,
                  rdf150, rdf151, rdf152]))

C:\Users\Mikael\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (24,32,34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### user ids

In [29]:
arcadeCabinetID = '"deb089c0-9be3-4b75-9b27-28963c77b10c"';
devPCID1 = '"7412a447-8177-48e9-82c5-cb31032f76a9"'
devPCID2 = '"7bb82da4-293a-4597-ba62-cc432b5898a5"'
devPCID3 = '"2036f6f4-50a1-4f94-8d4e-bfca814bbda8"'
devPCID4 = '"30d15b5f-2cf6-46c9-b187-c16a62f4259f"'

devPCID5 = '"7825d421-d668-4481-898a-46b51efe40f0"'
devPCID6 = '"97cdfb48-40e8-498e-afde-98f7290679f9"'
devPCID7 = '"3161ae02-6abd-4834-8208-686c4cd5363c"'
# possibly a tablet: a4d4b030-9117-4331-ba48-90dc05a7e65a

excludedIDs = ['null', devPCID1, devPCID2, devPCID3, devPCID4, devPCID5];